# Module 1 + Module 2 (no tuple)
The problem is formally defined as follows:
- a depot 0 where all vehicles are located in the beginning
- a set of customers $C = {1,...,N}$, we define locations $L = {0} + C$
- a demand $d_i$ for each customer $i \in C$ (the depot has no demand)
- a time window $[a_i,b_i]$ with $0<a_i<b_i$ for each location $i \in L$ (including the depot)
- a service time $s_i$ for each customer $i \in C$ (the depot has zero service time)
- costs $c_{ij}$ and time $t_{ij}$ for travelling from location $i$ to location $j$
- a homogeneous fleet $K = {1,...,|K|}$ of vehicles with load capacity $Q$

We want to find
- a route for each vehicle, i.e., an ordered sequence of customers
- with minimal total routing costs/time, such that
- each customer is visited exactly once
- the start times of the services have to be within the depot and customer time windows (potentially introducing waiting times between services),
- and the vehicle capacity is never exceeded

## Instance Configuration

In [172]:
import math
import random

### INSTANCE CONFIGURATION

numCustomers = 20
maxNumVehicles = 4

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100 # tmax
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)

## Instance Generation

In [173]:
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*range(1, maxNumVehicles + 1)]

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))
# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands, service times, and time window widths in the given range
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}
# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

## Output Visualisation

In [174]:
print("Customer Demands:")
for customer, demand in demands.items():
    print(f"Customer {customer}: Demand = {demand}")

print("\nCustomer Service Times:")
for customer, service_time in serviceTimes.items():
    print(f"Customer {customer}: Service Time = {service_time}")

print("\nCustomer Time Windows:")
for customer, window in timeWindows.items():
    print(f"Customer {customer}: Time Window = {window}")

print("\nTravel Times:")
for (i, j), time in list(travelTimes.items()):  # Displaying only first 10 for brevity
    print(f"Travel Time from {i} to {j}: {time}")


Customer Demands:
Customer 1: Demand = 4
Customer 2: Demand = 5
Customer 3: Demand = 2
Customer 4: Demand = 4
Customer 5: Demand = 5
Customer 6: Demand = 4
Customer 7: Demand = 3
Customer 8: Demand = 5
Customer 9: Demand = 5
Customer 10: Demand = 4
Customer 11: Demand = 2
Customer 12: Demand = 2
Customer 13: Demand = 2
Customer 14: Demand = 5
Customer 15: Demand = 2
Customer 16: Demand = 5
Customer 17: Demand = 4
Customer 18: Demand = 3
Customer 19: Demand = 4
Customer 20: Demand = 2
Customer 0: Demand = 0

Customer Service Times:
Customer 1: Service Time = 2
Customer 2: Service Time = 4
Customer 3: Service Time = 2
Customer 4: Service Time = 2
Customer 5: Service Time = 2
Customer 6: Service Time = 4
Customer 7: Service Time = 3
Customer 8: Service Time = 2
Customer 9: Service Time = 2
Customer 10: Service Time = 3
Customer 11: Service Time = 4
Customer 12: Service Time = 3
Customer 13: Service Time = 2
Customer 14: Service Time = 3
Customer 15: Service Time = 4
Customer 16: Service T

## Model Generation
### Basic Route Problem (without Load and Time constraint)
Decision variables:
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$

The objective is to minimize the total travel time/cost.

Constraints:
1. Each customer is visited exactly once, i.e., there is exactly one incoming connection and exactly one    nextCustomer connection.
2. At most $|K|$ vehicles can be used.
3. We can also compute a lower bound on the number of needed vehicles, that is the sum of all demands divided by the vehicle capacity.

$$
\text{min} \sum_{i,j \in \mathcal{L}} c_{ij}x_{ij}
$$

$$
\sum_{j \in \mathcal{C}} x_{ij} = 1 \quad \forall j \in \mathcal{C}
$$

$$
\sum_{j \in \mathcal{L}} x_{ij} = 1 \quad \forall i \in \mathcal{C}
$$

$$
\sum_{j \in \mathcal{C}} x_{0j} \leq |K|
$$

$$
\sum_{j \in \mathcal{C}} x_{0j} \geq \left\lceil \frac{\sum_{i \in \mathcal{C}} d_i}{Q} \right\rceil
$$


In [175]:
import gurobipy as gp
from gurobipy import GRB

# Function for calculate the number of vehicles needed for customers
def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

model = gp.Model("VRPTW")

# binary variables x(i,j): is 1 if some vehicle is going from node i to node j, 0 otherwise
x = model.addVars(connections, vtype=GRB.BINARY, name="x")

# objective function: minimize sum of connection costs
model.setObjective(x.prod(costs), GRB.MINIMIZE)

# all customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)

<gurobi.Constr *Awaiting Model Update*>

Then we can add Load and Time constraints in two different ways:

## Big - M Model

### Load

Decision Variable: $y_i$ for each location $i \in L$ to denote the vehicle load after picking up the demand at $i$.

Constraints:
- $y_0 = 0$
- $y_i + d_j \le y_j + Q(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$
- $y_i + d_j \geq y_j - M_{ij}(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$

### Time

Decision Variable: $z_i$ for each location $i \in L$ to denote the start time of the service at $i$ that has to be within its time window.

Constraints:
- $z_0 = 0$
- $z_i + s_i + t_{ij} \leq z_j + T(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$

In both case, subtours are eliminated since time/load has to monotonically increase along a route.

## Flow Model
Now, we extend the meaning of the variables used in the Big-M model. We do not associate resource states to customers anymore but instead to connections between customers.

### Load 

Decision variable: $y_{i,j}$ for each connection $(i, j)$ to denote the vehicle load after picking up the demand at $i$ and proceeding to location $j$.

Constraints:
 $$y_{0,j} = 0 \quad \forall j \in C$$
 $$\sum_{i \in L} y_{ij} + d_j = \sum_{i \in L} y_{ji} \quad \forall j \in C$$
 $$y_{ij} \geq d_ix_{ij} \quad \forall i \in C, j \in L, i \neq j$$
 $$y_{ij} \leq (Q - d_j)x_{ij} \quad \forall i \in C, j \in L, i \neq j$$

### Time
Decision variable: $z_{ij}$ for each connection $(i, j)$ to denote the start time of the service at $i$ (which must be within its time window) when immediately proceeding to location $j$.

Constraints:
 $$z_{0j} = 0 \quad \forall j \in C$$
 $$\sum_{i \in L}[z_{ij} + (s_i + t_{ij})x_{ij}] \leq \sum_{i \in L} z_{ji} \quad \forall j \in C$$
 $$z_{ij}\geq a_ix_{ij} \quad \forall i,j \in L, i \neq j$$
 $$z_{ij}\leq b_ix_{ij} \quad \forall i,j \in L, i \neq j$$

In [176]:
def addLoadConstraintsByBigM():

    y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
    y[0].UB = 0  # empty load at depot

    model.addConstrs(
        (
            y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM1",
    )
    model.addConstrs(
        (
            y[i] + demands[j]
            >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM2",
    )


def addLoadConstraintsByFlows():

    y = model.addVars(connections, lb=0, ub=vehicleCapacity, name="y")

    for i in customers:
        y[0, i].UB = 0

    model.addConstrs(
        (y.sum("*", j) + demands[j] == y.sum(j, "*") for j in customers),
        name="flowConservation",
    )
    model.addConstrs(
        (
            y[i, j] >= demands[i] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadLowerBound",
    )
    model.addConstrs(
        (
            y[i, j] <= (vehicleCapacity - demands[j]) * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadUpperBound",
    )


def addTimeConstraintsByBigM():

    z = model.addVars(locations, name="z")
    for i in locations:
        z[i].LB = timeWindows[i][0]
        z[i].UB = timeWindows[i][1]

    model.addConstrs(
        (
            z[i] + serviceTimes[i] + travelTimes[i, j]
            <= z[j]
            + (
                timeWindows[i][1]
                + serviceTimes[i]
                + travelTimes[i, j]
                - timeWindows[j][0]
            )
            * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="timeBigM",
    )


def addTimeConstraintsByFlows():

    z = model.addVars(connections, lb=0, name="z")

    for (i, j) in connections:
        z[i, j].UB = timeWindows[i][1]

    model.addConstrs(
        (
            gp.quicksum(
                z[i, j] + (serviceTimes[i] + travelTimes[i, j]) * x[i, j]
                for i in locations
                if (i, j) in connections
            )
            <= z.sum(j, "*")
            for j in customers
        ),
        name="flowConservation",
    )
    model.addConstrs(
        (
            z[i, j] >= timeWindows[i][0] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowStart",
    )
    model.addConstrs(
        (
            z[i, j] <= timeWindows[i][1] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowEnd",
    )


In [177]:
### MODEL CONFIGURATION
loadModelType = 1  # 1: big-M, 2: flow
timeModelType = 1  # 1: big-M, 2: flow

if loadModelType == 1:
    addLoadConstraintsByBigM()
elif loadModelType == 2:
    addLoadConstraintsByFlows()

if timeModelType == 1:
    addTimeConstraintsByBigM()
elif timeModelType == 2:
    addTimeConstraintsByFlows()

#model.params.Threads = 4
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1242 rows, 462 columns and 4438 nonzeros
Model fingerprint: 0xd6fbbf60
Variable types: 42 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+01, 1e+03]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]


Presolve removed 648 rows and 193 columns
Presolve time: 0.03s
Presolved: 594 rows, 269 columns, 3567 nonzeros
Variable types: 38 continuous, 231 integer (231 binary)
Found heuristic solution: objective 9794.0000000

Root relaxation: objective 4.960632e+03, 92 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4960.63179    0   36 9794.00000 4960.63179  49.4%     -    0s
H    0     0                    7996.0000000 4960.63179  38.0%     -    0s
H    0     0                    7630.0000000 4960.63179  35.0%     -    0s
*    0     0               0    5747.0000000 5747.00000  0.00%     -    0s

Cutting planes:
  Learned: 9
  Gomory: 6
  Cover: 7
  Implied bound: 21
  Clique: 2
  MIR: 16
  GUB cover: 1
  Zero half: 1
  RLT: 1
  Relax-and-lift: 6
  BQP: 2

Explored 1 nodes (128 simplex iterations) in 0.29 seconds (0.04 work units)
Thre

## Output Generation

In [178]:
if model.SolCount >= 1:

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (
                serviceTimes[currentCustomer]
                + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            )
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")

Solution contains 3 routes:
Route 0: 0 -> 4 (L:0, T:6) -> 8 (L:4, T:15) -> 12 (L:9, T:22) -> 19 (L:11, T:39) -> 2 (L:15, T:45) -> 11 (L:20, T:66) -> 0 (L:22/30, T:78)
Route 1: 0 -> 6 (L:0, T:7) -> 1 (L:4, T:24) -> 14 (L:8, T:38) -> 17 (L:13, T:45) -> 10 (L:17, T:63) -> 13 (L:21, T:69) -> 20 (L:23, T:72) -> 18 (L:25, T:78) -> 0 (L:28/30, T:84)
Route 2: 0 -> 7 (L:0, T:12) -> 5 (L:3, T:17) -> 3 (L:8, T:22) -> 15 (L:10, T:25) -> 9 (L:12, T:36) -> 16 (L:17, T:74) -> 0 (L:22/30, T:85)
Unvisited customers: 

T[i] tempo di arrivo al nodo i

z[i] tempo in cui puo iniziare il servizio in i

# Module 1 + Module 2 

In [164]:
import gurobipy as gp
from gurobipy import GRB
import math
import random

### INSTANCE CONFIGURATION
numCustomers = 10
maxNumVehicles = 3

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*range(1, maxNumVehicles + 1)]

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))
# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands, service times, and time window widths in the given range
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}
# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

# Define the restricted tuples R
R = [
    (1, 7, 3)
]

# create model for Capacitated Vehicle Routing Problem instance
model = gp.Model("VRPTW")

# binary variables x(i,j): is 1 if some vehicle is going from node i to node j, 0 otherwise
x = model.addVars(connections, vtype=GRB.BINARY, name="x")

# objective function: minimize sum of connection costs
model.setObjective(x.prod(costs), GRB.MINIMIZE)

# all customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)
y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
y[0].UB = 0  # empty load at depot

model.addConstrs(
    (
        y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM1",
)
model.addConstrs(
    (
        y[i] + demands[j]
        >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM2",
)

z = model.addVars(locations, name="z")
for i in locations:
    z[i].LB = timeWindows[i][0]
    z[i].UB = timeWindows[i][1]

model.addConstrs(
    (
        z[i] + serviceTimes[i] + travelTimes[i, j]
        <= z[j]
        + (
            timeWindows[i][1]
            + serviceTimes[i]
            + travelTimes[i, j]
            - timeWindows[j][0]
        )
        * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="timeBigM",
)

# Add the new constraint for the set R
for (i, j, l) in R:
    model.addConstr(
        gp.quicksum(x[i, k] for k in customers if k != i and k != j) + 
        gp.quicksum(x[k, j] for k in customers if k != i and k != j) - 
        1 >= x[i, j] + x[j, l] - 1,
        name=f"no_shared_vehicle_{i}_{j}_{l}"
    )

model.params.Threads = 4
model.optimize()
if model.SolCount >= 1:

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (
                serviceTimes[currentCustomer]
                + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            )
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")


Set parameter Threads to value 4
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 323 rows, 132 columns and 1138 nonzeros
Model fingerprint: 0xfec85e75
Variable types: 22 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+01, 1e+03]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]
Presolve removed 178 rows and 61 columns
Presolve time: 0.01s
Presolved: 145 rows, 71 columns, 837 nonzeros
Variable types: 16 continuous, 55 integer (55 binary)
Found heuristic solution: objective 5986.0000000

Root relaxation: objective 4.026464e+03, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod